In [ ]:
from __future__ import annotations

# Install missing package for Gemini / Google Generative AI client
%pip install google-generativeai
%pip install --pre -U langchain langchain-openai

# ============================================================
# IMPORTS
# ============================================================

import json
import logging
import os
import time
from decimal import Decimal
from enum import Enum

import fitz  # PyMuPDF
import google.generativeai as genai
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel, ConfigDict, Field, field_validator
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [ ]:
# ============================================================
# CONFIGURACIÓN
# ============================================================

# Cargar variables de entorno
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Configurar Gemini
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

# Nombre del archivo a procesar (sin extensión)
DOCUMENT_ID = 'DOC_TECNICO_LICITACION_002_multi_item'

# Construir rutas
file = os.path.join(os.path.dirname(os.getcwd()), 'file_to_test', f'{DOCUMENT_ID}.pdf')
json_file = os.path.join(os.path.dirname(os.getcwd()), 'file_to_test', f'{DOCUMENT_ID}.json')
print(f'Procesando archivo: {file}')

In [ ]:
# ============================================================
# SCHEMAS PYDANTIC - TIPOS REUTILIZABLES
# ============================================================

class NumericRange(BaseModel):
    """Rango numérico genérico con unidad opcional."""
    model_config = ConfigDict(extra='forbid')

    min: Decimal = Field(..., description="Valor mínimo")
    max: Decimal = Field(..., description="Valor máximo")
    unidad: str | None = Field(None, description="Unidad (V, A, Hz, °C, kVA, etc.)")

    @field_validator('max')
    @classmethod
    def _min_le_max(cls, v, info):
        min_val = info.data.get('min')
        if min_val is not None and v < min_val:
            raise ValueError('max debe ser >= min')
        return v


class Dimensiones(BaseModel):
    model_config = ConfigDict(extra='forbid')

    ancho_mm: Decimal | None = Field(None, description="Ancho en mm")
    alto_mm: Decimal | None = Field(None, description="Alto en mm")
    profundidad_mm: Decimal | None = Field(None, description="Profundidad en mm")


class InterruptorSpec(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tipo: str | None = Field(None, description="Tipo de interruptor")
    calibre_a: Decimal | None = Field(None, description="Calibre en amperes (A)")


class FaseTipo(str, Enum):
    monofasica = "monofásica"
    bifasica = "bifásica"
    trifasica = "trifásica"


class ConexionNeutroTierra(str, Enum):
    TT = "TT"
    TN = "TN"
    IT = "IT"
    otro = "otro"


class ServicioTipo(str, Enum):
    continuo = "continuo"
    intermitente = "intermitente"
    otro = "otro"


class AlimentacionAlternativaTipo(str, Enum):
    interna = "interna"
    externa = "externa"

In [ ]:
# ============================================================
# SCHEMAS PYDANTIC - SECCIONES DEL SISTEMA
# ============================================================

class Generales(BaseModel):
    model_config = ConfigDict(extra='forbid')

    codigo_producto: str | None = None
    marca: str | None = None
    modelo: str | None = None
    normas_fabricacion: str | None = Field(None, description="Normas / estándares aplicables")
    origen: str | None = Field(None, description="País o región de fabricación")
    tipo: str | None = Field(None, description="Familia / categoría del equipo")
    apto_baterias_pb_y_nicd: bool | None = Field(None, description="Apto para Pb-Ca/Pb-Ac y Ni-Cd")


class CondicionesAmbientales(BaseModel):
    model_config = ConfigDict(extra='forbid')

    temperatura_max_c: Decimal | None = Field(None, description="Temperatura máxima de operación (°C)")
    temperatura_min_c: Decimal | None = Field(None, description="Temperatura mínima de operación (°C)")
    altura_snm_m: Decimal | None = Field(None, description="Altura sobre el nivel del mar (m)")
    humedad_relativa_max_pct: Decimal | None = Field(None, description="Humedad relativa máx. sin condensación (%)")
    instalacion: str | None = Field(None, description="Interior/Exterior/Armario/etc.")
    tipo_servicio: ServicioTipo | None = None


class Alimentacion(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tipo: FaseTipo | None = Field(None, description="Tipo de fase")
    tension_v: Decimal | None = Field(None, description="Tensión nominal (V)")
    rango_tension_entrada: NumericRange | None = Field(None, description="Rango de tensión admisible (V)")
    frecuencia_hz: Decimal | None = Field(None, description="Frecuencia nominal (Hz)")
    rango_frecuencia_entrada: NumericRange | None = Field(None, description="Rango de frecuencia admisible (Hz)")
    conexion_neutro_tierra: ConexionNeutroTierra | None = Field(None, description="Esquema de conexión a tierra")
    conductor_pe_independiente: bool | None = Field(None, description="PE independiente del neutro")
    isc_punto_conexion: Decimal | None = Field(None, description="Corriente de cortocircuito en punto de conexión (A o kA)")
    interruptor_acometida: InterruptorSpec | None = Field(None, description="Interruptor de acometida")
    potencia_transformador_entrada_kva: Decimal | None = Field(None, description="Potencia del transformador de entrada (kVA)")
    corriente_conexion_transformador_a: Decimal | None = Field(None, description="Corriente de conexión del transformador (A)")


class Salida(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tension_nominal_v: Decimal | None = Field(None, description="Tensión nominal de salida fija (V)")
    corriente_nominal_a: Decimal | None = Field(None, description="Corriente nominal de salida fija (A)")
    maxima_corriente_consumos_a: Decimal | None = Field(None, description="Máxima corriente a consumos (A)")

    # Campos para salidas ajustables
    tension_ajustable: NumericRange | None = Field(None, description="Rango de tensión ajustable (V)")
    corriente_ajustable: NumericRange | None = Field(None, description="Rango de corriente ajustable (A)")


class ModosCarga(BaseModel):
    model_config = ConfigDict(extra='forbid')

    manual_automatico: str | None = Field(None, description="Modo de operación: manual/automático")
    carga_excepcional: bool | None = Field(None, description="Disponibilidad de carga excepcional")


class RangoTensionPorQuimica(BaseModel):
    model_config = ConfigDict(extra='forbid')

    nicd: NumericRange | None = Field(None, description="Rango de tensión para Ni-Cd (V)")
    pb_ca: NumericRange | None = Field(None, description="Rango de tensión para Pb-Ca (V)")


class CargaBaterias(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tension_flote: NumericRange | None = Field(None, description="Tensión de flote (rango de ajuste, V)")
    estabilizacion_tension_flote: str | None = Field(None, description="Estabilización de tensión de flote (ej., ±1%)")
    tension_fondo: NumericRange | None = Field(None, description="Tensión de fondo (rango de ajuste, V)")
    modos_carga: ModosCarga | None = None
    regulador_diodos: bool | None = Field(None, description="Regulación mediante cadena de diodos")
    rango_tension_salida_consumo: RangoTensionPorQuimica | None = Field(None, description="Rango de tensión de salida a consumo por química")
    deteccion_polo_tierra: bool | None = Field(None, description="Detección de polo a tierra")
    interruptor_salida_consumo: InterruptorSpec | None = Field(None, description="Interruptor de salida a consumo")
    interruptor_salida_baterias: InterruptorSpec | None = Field(None, description="Interruptor de salida a baterías")
    sistema_rectificacion: str | None = Field(None, description="Tecnología del rectificador")
    ripple_con_baterias: str | None = Field(None, description="Nivel de rizado con baterías (mVpp o %)")
    ripple_sin_baterias: str | None = Field(None, description="Nivel de rizado sin baterías (mVpp o %)")


class Gabinete(BaseModel):
    model_config = ConfigDict(extra='forbid')

    material: str | None = None
    acceso: str | None = Field(None, description="Acceso: frontal/lateral/trasero")
    grado_proteccion: str | None = Field(None, description="IP/NEMA")
    espesor_chapa_estructura_mm: Decimal | None = Field(None, description="Espesor de chapa (mm)")
    tipo_pintura: str | None = None
    color: str | None = Field(None, description="Código RAL u otro")
    espesor_pintura_micras: Decimal | None = Field(None, description="Espesor de pintura (μm)")
    dimensiones: Dimensiones | None = Field(None, description="Dimensiones (mm)")


class Otros(BaseModel):
    model_config = ConfigDict(extra='forbid')

    ventilacion: str | None = Field(None, description="Natural/forzada")
    rectificador: str | None = Field(None, description="Topología o modelo del rectificador")


class Protecciones(BaseModel):
    model_config = ConfigDict(extra='forbid')

    sobretensiones_cc: bool | None = Field(None, description="Protección contra sobretensiones en CC")
    sobretensiones_ca: bool | None = Field(None, description="Protección contra sobretensiones en CA")
    cortocircuito: bool | None = Field(None, description="Protección contra cortocircuito")
    sobrecarga: bool | None = Field(None, description="Protección contra sobrecarga")
    lvd: bool | None = Field(None, description="Desconexión por baja tensión (LVD)")


class Alarmas(BaseModel):
    model_config = ConfigDict(extra='forbid')

    falla_red: bool | None = None
    alta_tension_baterias: bool | None = None
    baja_tension_baterias: bool | None = None
    tension_baterias_critica: bool | None = Field(None, description="Batería descargada")
    alta_tension_consumos: bool | None = None
    baja_tension_consumos: bool | None = None
    tension_consumo_critica: bool | None = None
    falla_rectificador: bool | None = None
    alta_corriente_rectificador: bool | None = None
    alta_corriente_baterias: bool | None = None
    alta_corriente_consumos: bool | None = None
    polo_positivo_tierra: bool | None = None
    polo_negativo_tierra: bool | None = None
    fallo_ventilador: bool | None = Field(None, description="Si corresponde")
    obstruccion_filtros: bool | None = Field(None, description="Si corresponde")
    bateria_en_descarga: bool | None = None
    averia_red_salida: bool | None = Field(None, description="Falla de aislación")
    apertura_interruptores: bool | None = None
    fallo_bateria_supervision: bool | None = Field(None, description="Si hay sistema de supervisión")
    temperatura_termostato: bool | None = None
    cargador_modo_fondo: bool | None = None


class Senalizaciones(BaseModel):
    model_config = ConfigDict(extra='forbid')

    red_ok: bool | None = Field(None, description="Presencia de tensión alterna en redes")
    bateria_carga_flotacion: bool | None = None
    bateria_carga_rapida: bool | None = None
    bateria_carga_excepcional: bool | None = None
    ventiladores_funcionando: bool | None = Field(None, description="Si aplica")
    carga_por_red_alternativa: bool | None = None
    falla: bool | None = None
    tension_bateria: bool | None = None
    tension_consumos: bool | None = None
    corriente_baterias: bool | None = None
    corriente_consumos: bool | None = None
    alarmas_discriminadas: bool | None = None


class AparatosDeMedida(BaseModel):
    model_config = ConfigDict(extra='forbid')

    unidad_digital_centralizada: bool | None = None
    protocolo_comunicacion: str | None = None
    puerto_comunicacion: str | None = None
    mide_corrientes_entrada: bool | None = None
    mide_tensiones_entrada: bool | None = None
    mide_corriente_salida_rectificador: bool | None = None
    mide_corriente_carga_baterias: bool | None = None
    mide_tension_salida_rectificador: bool | None = None
    mide_tension_baterias: bool | None = None
    mide_tension_consumos: bool | None = None
    mide_corriente_descarga_baterias: bool | None = None


class ResistenciasCalefactoras(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tension_alimentacion_v: Decimal | None = Field(None, description="V")
    potencia_w: Decimal | None = Field(None, description="W")
    llave_termomagnetica_independiente: bool | None = Field(None, description="Llave dedicada")
    alimentacion: AlimentacionAlternativaTipo | None = Field(None, description="Interna o externa")


class CablesPotenciaAuxiliares(BaseModel):
    model_config = ConfigDict(extra='forbid')

    tension_aislacion_v: Decimal | None = Field(None, description="Tensión de aislación (V)")
    material: str | None = None
    baja_emision_halogenos: bool | None = None


class CaracteristicasAccesoriosAuxiliares(BaseModel):
    model_config = ConfigDict(extra='forbid')

    panel_control: str | None = Field(None, description="Ajuste y configuración")
    resistencias_calefactoras: ResistenciasCalefactoras | None = None
    cables_potencia_auxiliares: CablesPotenciaAuxiliares | None = None
    bornes_reserva: int | None = Field(None, description="Cantidad de bornes de reserva")
    nivel_ruido_db_a: Decimal | None = Field(None, description="Nivel de ruido a 1 m y 100% carga (dB(A))")
    placas_identificacion: bool | None = None
    chapa_caracteristicas: bool | None = None


class InspeccionesEnsayos(BaseModel):
    model_config = ConfigDict(extra='forbid')

    resistencia_aislacion: bool | None = Field(None, description="Medición de resistencia de aislación")
    ensayo_dieletrico: bool | None = Field(None, description="Ensayo dieléctrico de potencia/control (excluye electrónicos)")
    resistencia_aislacion_post: bool | None = Field(None, description="Medición posterior al ensayo dieléctrico")
    elementos_mecanicos_enclavamientos: bool | None = Field(None, description="Funcionamiento y enclavamientos")
    tolerancia_tension_salida: bool | None = None
    nivel_rizado: bool | None = None
    verificacion_valores_salida_control: bool | None = Field(None, description="Verificación eléctrica en condiciones de carga")
    verificacion_tensiones_carga: bool | None = Field(None, description="Verifica flote/rápida/excepcional")
    variacion_tension_alimentacion: bool | None = Field(None, description="No varía en límites de alimentación")
    variacion_carga_0_a_100: bool | None = Field(None, description="No varía del 0% al 100% de carga")
    verificacion_carga_bateria: bool | None = None
    contenido_armonicos_alimentacion: bool | None = None
    ensayo_plena_carga_48h: bool | None = Field(None, description="Ensayo a plena carga hasta régimen estable")
    ensayo_rendimiento: bool | None = None
    reparto_intensidades_paralelo: bool | None = Field(None, description="Para cargadores en paralelo")
    prueba_senalizacion_alarma: bool | None = None
    prueba_funcional_total: bool | None = Field(None, description="Todas las condiciones de operación")
    capacidad_corriente_cortocircuito: bool | None = Field(None, description="Capacidad para abrir el mayor interruptor de salida")


class Garantia(BaseModel):
    model_config = ConfigDict(extra='forbid')

    descripcion: str | None = Field(None, description="Términos de garantía")
    meses: int | None = Field(None, description="Duración en meses")


class SistemaCargadorRectificador(BaseModel):
    """Esquema completo del sistema cargador/rectificador + baterías."""
    model_config = ConfigDict(extra='forbid')

    generales: Generales | None = None
    condiciones_ambientales: CondicionesAmbientales | None = None
    alimentacion: Alimentacion | None = None
    salida: Salida | None = None
    carga_baterias: CargaBaterias | None = None
    gabinete: Gabinete | None = None
    otros: Otros | None = None
    protecciones: Protecciones | None = None
    alarmas: Alarmas | None = None
    senalizaciones: Senalizaciones | None = None
    aparatos_medida: AparatosDeMedida | None = None
    caracteristicas_accesorios: CaracteristicasAccesoriosAuxiliares | None = None
    inspecciones_ensayos: InspeccionesEnsayos | None = None
    garantia: Garantia | None = None


# ============================================================
# SCHEMAS PARA LICITACIÓN CON MÚLTIPLES ITEMS
# ============================================================

class ItemLicitado(BaseModel):
    """Representa un item individual dentro de la licitación."""
    model_config = ConfigDict(extra='forbid')

    numero_item: int | None = Field(None, description="Número de ítem en la licitación (1, 2, 3, 4, etc.)")
    cantidad: int | None = Field(None, description="Cantidad requerida de unidades")
    descripcion: str | None = Field(None, description="Descripción del item")

    # Especificaciones específicas que varían por item
    alimentacion: Alimentacion | None = Field(None, description="Alimentación específica de este item")
    salida: Salida | None = Field(None, description="Salida específica de este item")
    sistema_control: str | None = Field(None, description="Sistema de control específico")

    # Marca y modelo si se especifican por item
    marca: str | None = None
    modelo: str | None = None


class LicitacionCompleta(BaseModel):
    """Esquema para licitaciones con múltiples items y especificaciones comunes."""
    model_config = ConfigDict(extra='forbid')

    # Especificaciones comunes a todos los items
    especificaciones_comunes: SistemaCargadorRectificador | None = Field(
        None,
        description="Especificaciones técnicas que aplican a todos los items de la licitación"
    )

    # Lista de items individuales
    items: list[ItemLicitado] = Field(
        default_factory=list,
        description="Lista de items individuales licitados con sus especificaciones particulares"
    )

In [ ]:
# ============================================================
# FUNCIONES DE PROCESAMIENTO
# ============================================================

def read_pdf_from_file(file_path) -> list[dict]:
    """Lee PDF desde archivo y extrae texto con metadatos de página."""
    try:
        pdf_document = fitz.open(file_path)

        pages_data = []
        for page_num in range(pdf_document.page_count):
            page = pdf_document[page_num]
            page_text = page.get_text()
            pages_data.append({
                'text': page_text,
                'page_number': page_num + 1,
                'total_pages': pdf_document.page_count
            })

        pdf_document.close()
        return pages_data

    except Exception as e:
        print(f"Error reading PDF from file: {e}")
        return None


def chunk_text(self, pages_data: list[dict], chunk_size: int = 1000, chunk_overlap: int = 200) -> list[dict]:
    """Divide el texto en chunks con metadatos semánticos."""
    if not pages_data:
        return []

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks_with_metadata = []
    for page_data in pages_data:
        if not page_data['text'].strip():
            continue

        chunks = text_splitter.split_text(page_data['text'])
        for i, chunk in enumerate(chunks):
            chunk_metadata = {
                'page_number': page_data['page_number'],
                'total_pages': page_data['total_pages'],
                'chunk_index': i
            }
            doc = {}
            doc['page_content'] = chunk
            doc['metadata'] = chunk_metadata
            chunks_with_metadata.append(doc)

    return chunks_with_metadata

In [ ]:
# ============================================================
# SETUP LLM Y PROMPTS
# ============================================================

# Inicializar modelo
llm = ChatOpenAI(model="gpt-4.1", temperature=0)
fast_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt para clasificación técnica/no-técnica
classification_prompt = ChatPromptTemplate.from_template("""
You are a technical content classifier for electrical and electronics
  tender documents. Your task is to categorize each text chunk as either
  "TECHNICAL" or "NON-TECHNICAL".

  **TECHNICAL** chunks contain:
  - Product specifications, technical requirements, or performance
  criteria
  - Equipment descriptions, models, or part numbers
  - Electrical parameters (voltage, current, power, frequency, etc.)
  - Technical standards, certifications, or compliance requirements
  - Installation, configuration, or operational procedures
  - System architecture, wiring diagrams, or technical drawings
  - Quality assurance, testing, or measurement specifications
  - Materials, components, or manufacturing details
  - Technical service requirements or maintenance procedures

  **NON-TECHNICAL** chunks contain:
  - Administrative procedures, legal terms, or contractual conditions
  - Payment terms, delivery schedules, or commercial conditions
  - General company information or background
  - Bidding instructions or submission requirements
  - Insurance, liability, or legal compliance (non-technical)
  - Project timelines or administrative milestones
  - General introductory or concluding statements
  - Contact information or communication protocols

  **Instructions:**
  1. Read the text chunk carefully
  2. Respond with only "TECHNICAL" or "NON-TECHNICAL"
  3. When in doubt, err on the side of "TECHNICAL" to ensure only
  clearly technical content passes through

  **Text chunk to classify:**
  {text_chunk}
  **Classification:**
"""
)

# Prompt para extracción estructurada (usado por Langchain chunking)
extraction_prompt = ChatPromptTemplate.from_template("""
You are an expert data extraction system for electrical and
  electronics tender documents.

  Extract all relevant technical information from the provided text
  chunk and structure it according to the SistemaCargadorRectificador
  schema.

  **Instructions:**
  1. Extract only information that is explicitly stated in the text
  2. Use appropriate data types (Decimal for numbers, enums for
  categorical values)
  3. Leave fields as None if information is not present or unclear
  4. For numeric ranges, extract both min and max values with units
  5. Be precise with technical specifications and measurements

  **Key categories to look for:**
  - Product specifications (marca, modelo, código)
  - Environmental conditions (temperatura, humedad, altitud)
  - Electrical parameters (tensión, corriente, potencia, frecuencia)
  - Technical standards and certifications
  - Installation and operational requirements
  - Physical dimensions and materials
  - Protection systems and alarms
  - Measurement instruments and communication protocols

  **Text chunk:**
  {text_chunk}

  Extract and structure all available technical data from this text.
"""
)

# Prompt para extracción de múltiples items (usado por Gemini)
multi_item_extraction_prompt = ChatPromptTemplate.from_template("""
You are an expert data extraction system for electrical and electronics tender documents.

Extract ALL items from the tender document and structure them according to the LicitacionCompleta schema.

**CRITICAL INSTRUCTIONS:**
1. This tender document contains MULTIPLE items (typically Item 1, Item 2, Item 3, Item 4, etc.)
2. Each item has its own specifications (quantity, input voltage, output current, etc.)
3. There are also COMMON specifications that apply to ALL items
4. You MUST extract each item separately into the "items" array
5. Extract common specifications into "especificaciones_comunes"

**For each individual item extract:**
- numero_item: The item number (1, 2, 3, 4, etc.)
- cantidad: Quantity required
- descripcion: Brief description of the item
- alimentacion: Specific input specifications (voltage, phase type)
- salida: Specific output specifications
  - For FIXED outputs, use tension_nominal_v and corriente_nominal_a
  - For ADJUSTABLE outputs (0-150V, 0-15A, etc.), use tension_ajustable and corriente_ajustable with NumericRange
- sistema_control: Control system details if specified

**For common specifications (especificaciones_comunes), extract:**
- All technical requirements that apply to ALL items
- Environmental conditions
- Common electrical specifications
- Standards and certifications
- Protections, alarms, signaling
- Testing and inspection requirements
- Warranty terms

**Text to process:**
{text_chunk}

Extract all items and common specifications from this tender document.
"""
)

query_generator_prompt = ChatPromptTemplate.from_template("""
                                                          
You are an expert assistant for generating PostgreSQL SQL queries. Your task is to convert JSON objects into SQL SELECT queries that search for products in a rectifier database following these strict rules:

## Database Structure

The database has the following main tables and their EXACT column names:

### productos table
Columns: id, codigo, marca, modelo, tension_nominal, corriente_nominal, regulador_diodos, origen, tipo, created_at, updated_at

### especificaciones table
Columns: id, producto_id, normas_fabricacion, apto_pb_ac, apto_ni_cd, temperatura_maxima, temperatura_minima, altura_snm, humedad_relativa_max, tipo_instalacion, tipo_servicio, ventilacion, tipo_rectificador, nivel_ruido, rendimiento_minimo, proteccion_sobretension, proteccion_cortocircuito, proteccion_sobrecarga, ripple_con_baterias, ripple_sin_baterias, tension_flote_min, tension_flote_max, tension_fondo_min, tension_fondo_max, modo_manual_automatico, modo_carga_excepcional, regulador_diodos_carga, rango_salida_nicd, rango_salida_pbca, deteccion_polo_tierra

### alimentacion table
Columns: id, producto_id, tipo, tension, rango_tension, frecuencia, rango_frecuencia, conexion_neutro, conductor_pe_independiente, corriente_cortocircuito, tipo_interruptor_acometida, potencia_transformador, corriente_conexion_transformador

### salida table
Columns: id, producto_id, tension_nominal, corriente_nominal, maxima_corriente_consumos, tipo_interruptor_consumo, tipo_interruptor_baterias, sistema_rectificacion

### gabinete table
Columns: id, producto_id, material, acceso, grado_proteccion, espesor_chapa, tipo_pintura, color, espesor_pintura, ancho, alto, profundidad, peso

### aparatos_medida table
Columns: id, producto_id, unidad_digital_centralizada, protocolo_comunicacion, puerto_comunicacion, medicion

### accesorios table
Columns: id, producto_id, panel_control, resistencias_calefactoras, tension_resistencias, potencia_resistencias, cables_incluidos, tension_aislacion_cables, material_cables, baja_emision_halogenos, bornes_reserva, placas_identificacion, chapa_caracteristicas

### garantia table
Columns: id, producto_id, meses, condiciones

## Generation Rules

1. **CRITICAL - Ignore null values**: If a field has a null value, is explicitly set to null, or is not present in the JSON, DO NOT include it in the WHERE clause at all. Never use "IS NULL" comparisons in the WHERE clause.

2. **CRITICAL - Validate column existence**: Before adding any condition to the WHERE clause, verify that the column name exists in the exact table structure provided above. If a JSON field name doesn't match any column in any table, skip it entirely.

3. **CRITICAL - Multiple products = Multiple queries with UNION ALL**: 
   - If the JSON contains an ARRAY of products/items, create ONE separate SELECT query for EACH item
   - Join ALL queries using UNION ALL
   - Each query gets a sequential origen_consulta number (1, 2, 3, etc.)
   - Even if products share common fields, create separate queries for each
   - Example: If JSON has 4 products, generate 4 SELECT queries joined by UNION ALL
   - **This is MANDATORY - never combine multiple products into a single query**

4. **JOIN only when necessary**: Only include LEFT JOIN for tables that have active filters (non-null values that exist in that table)

5. **Base structure for SINGLE product**:
   ```sql
   SELECT DISTINCT p.*, 1 AS origen_consulta
   FROM productos p
   [necessary JOINs only for tables with active conditions]
   WHERE [conditions - only for non-null values]
   ```

6. **Base structure for MULTIPLE products (ALWAYS use UNION ALL)**:
   ```sql
   SELECT DISTINCT p.*, 1 AS origen_consulta
   FROM productos p
   [JOINs]
   WHERE [conditions for first product]
   UNION ALL
   SELECT DISTINCT p.*, 2 AS origen_consulta
   FROM productos p
   [JOINs]
   WHERE [conditions for second product]
   UNION ALL
   SELECT DISTINCT p.*, 3 AS origen_consulta
   FROM productos p
   [JOINs]
   WHERE [conditions for third product]
   UNION ALL
   SELECT DISTINCT p.*, 4 AS origen_consulta
   FROM productos p
   [JOINs]
   WHERE [conditions for fourth product]
   ```

7. **Detecting multiple products - CRITICAL**: Look for these patterns in the JSON:
    - Array of objects: `[{{product1}}, {{product2}}, {{product3}}, {{product4}}]`
    - Multiple numbered keys: `{{"producto_1": {{...}}, "producto_2": {{...}}, "producto_3": {{...}}, "producto_4": {{...}}`
    - Array inside a key: `{{"productos": [{{...}}, {{...}}, {{...}}, {{...}}]}}`
    - **If you find ANY array or multiple product objects, you MUST create separate queries for EACH item using UNION ALL**
    - Count the items carefully - 4 items = 4 separate SELECT queries

8. **Comparisons**:
   - Numeric fields: use exact comparison (=) or ranges if the field ends in _min/_max
   - Text fields: use ILIKE '%value%' for partial case-insensitive search
   - Boolean fields: use = true or = false

9. **Table aliases**:
   - productos → p
   - especificaciones → e
   - alimentacion → a
   - salida → s
   - gabinete → g
   - aparatos_medida → am
   - accesorios → ac
   - garantia → ga

10. **Handling nested JSON structure**: The JSON may have nested structure. Flatten this structure and find the corresponding field in the database tables by matching the field name to the exact column names listed above.

11. **Field matching priority**: When matching JSON field names to database columns:
   - First, try exact match (case-insensitive)
   - If no exact match, try to find the most similar column name
   - If no reasonable match exists in any table, skip that field entirely

12. **ALWAYS include origen_consulta**: Every SELECT statement MUST include the origen_consulta column. Start numbering from 1 and increment for each UNION ALL query.

## Response Format

Respond ONLY with the SQL query, without additional explanations, without markdown code blocks, without text before or after. Just pure SQL.

## Examples

**Example 1 - Single product:**
```json
{{
  "codigo": "RECT-001",
  "marca": "SERVELEC",
  "temperatura_maxima": 45
}}
```

**Expected Output:**
```sql
SELECT DISTINCT p.*, 1 AS origen_consulta
FROM productos p
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE p.codigo ILIKE '%RECT-001%'
  AND p.marca ILIKE '%SERVELEC%'
  AND e.temperatura_maxima = 45
```

**Example 2 - Three products with UNION ALL:**
```json
[
  {{
    "tension_nominal": 110,
    "corriente_nominal": 50,
    "frecuencia": 60
  }},
  {{
    "tension_nominal": 110,
    "corriente_nominal": 40,
    "frecuencia": 50
  }},
  {{
    "tension_nominal": 48,
    "corriente_nominal": 30,
    "frecuencia": 40
  }}
]
```

**Expected Output:**
```sql
SELECT DISTINCT p.*, 1 AS origen_consulta
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
WHERE s.tension_nominal = 110
  AND s.corriente_nominal = 50
  AND s.frecuencia = 60
UNION ALL
SELECT DISTINCT p.*, 2 AS origen_consulta
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
WHERE s.tension_nominal = 110
  AND s.corriente_nominal = 40
  AND s.frecuencia = 50
UNION ALL
SELECT DISTINCT p.*, 3 AS origen_consulta
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
WHERE s.tension_nominal = 48
  AND s.corriente_nominal = 30
  AND s.frecuencia = 40
```

**Example 3 - Four products with shared and different fields:**
```json
[
  {{
    "tipo": "Autorregulado",
    "temperatura_maxima": 45,
    "tipo_alimentacion": "trifásica",
    "tension": "380",
    "tension_nominal_salida": 110,
    "corriente_nominal_salida": 50
  }},
  {{
    "tipo": "Autorregulado",
    "temperatura_maxima": 45,
    "tipo_alimentacion": "trifásica",
    "tension": "380",
    "tension_nominal_salida": 110,
    "corriente_nominal_salida": 40
  }},
  {{
    "tipo": "Autorregulado",
    "temperatura_maxima": 45,
    "tipo_alimentacion": "monofásica",
    "tension": "220",
    "tension_nominal_salida": 110,
    "corriente_nominal_salida": 30
  }},
  {{
    "tipo": "Autorregulado",
    "temperatura_maxima": 45,
    "tipo_alimentacion": "monofásica",
    "tension": "220"
  }}
]
```

**Expected Output:**
```sql
SELECT DISTINCT p.*, 1 AS origen_consulta
FROM productos p
LEFT JOIN especificaciones e ON p.id = e.producto_id
LEFT JOIN alimentacion a ON p.id = a.producto_id
LEFT JOIN salida s ON p.id = s.producto_id
WHERE p.tipo ILIKE '%Autorregulado%'
  AND e.temperatura_maxima = 45
  AND a.tipo ILIKE '%trifásica%'
  AND a.tension ILIKE '%380%'
  AND s.tension_nominal = 110
  AND s.corriente_nominal = 50
UNION ALL
SELECT DISTINCT p.*, 2 AS origen_consulta
FROM productos p
LEFT JOIN especificaciones e ON p.id = e.producto_id
LEFT JOIN alimentacion a ON p.id = a.producto_id
LEFT JOIN salida s ON p.id = s.producto_id
WHERE p.tipo ILIKE '%Autorregulado%'
  AND e.temperatura_maxima = 45
  AND a.tipo ILIKE '%trifásica%'
  AND a.tension ILIKE '%380%'
  AND s.tension_nominal = 110
  AND s.corriente_nominal = 40
UNION ALL
SELECT DISTINCT p.*, 3 AS origen_consulta
FROM productos p
LEFT JOIN especificaciones e ON p.id = e.producto_id
LEFT JOIN alimentacion a ON p.id = a.producto_id
LEFT JOIN salida s ON p.id = s.producto_id
WHERE p.tipo ILIKE '%Autorregulado%'
  AND e.temperatura_maxima = 45
  AND a.tipo ILIKE '%monofásica%'
  AND a.tension ILIKE '%220%'
  AND s.tension_nominal = 110
  AND s.corriente_nominal = 30
UNION ALL
SELECT DISTINCT p.*, 4 AS origen_consulta
FROM productos p
LEFT JOIN especificaciones e ON p.id = e.producto_id
LEFT JOIN alimentacion a ON p.id = a.producto_id
WHERE p.tipo ILIKE '%Autorregulado%'
  AND e.temperatura_maxima = 45
  AND a.tipo ILIKE '%monofásica%'
  AND a.tension ILIKE '%220%'
```

**CRITICAL NOTE for Example 3**: This JSON has 4 products in an array, so it generates 4 separate SELECT queries joined by UNION ALL. Even though products 1-2 share fields and products 3-4 share fields, they are NEVER combined - each product gets its own query.

Now process the following JSON and generate the corresponding SQL query: {json_input}""")

query_sanitizer_prompt = ChatPromptTemplate.from_template("""

You are an expert SQL query optimizer. Your task is to take an existing PostgreSQL SELECT query and modify specific comparison operators and translate English text values to Spanish according to the rules below.

## Optimization Rules

Apply these transformations to the WHERE clause:

### 1. Temperature and Humidity Fields (especificaciones table)
Replace exact comparisons with range comparisons:

- **temperatura_maxima**: Change `= value` to `<= value`
  - Original: `e.temperatura_maxima = 45`
  - Optimized: `e.temperatura_maxima <= 45`

- **temperatura_minima**: Change `= value` to `>= value`
  - Original: `e.temperatura_minima = -10`
  - Optimized: `e.temperatura_minima >= -10`

- **humedad_relativa_max**: Change `= value` to `<= value`
  - Original: `e.humedad_relativa_max = 85`
  - Optimized: `e.humedad_relativa_max <= 85`

### 2. Replace text comparisons with similarity function
Replace text comparisons (both ILIKE and =) with the similarity function, but ONLY for text/VARCHAR fields:

**TEXT FIELDS that should use similarity:**
- productos: codigo, marca, modelo, regulador_diodos, origen, tipo
- especificaciones: normas_fabricacion, tipo_instalacion, tipo_servicio, ventilacion, tipo_rectificador, nivel_ruido, proteccion_sobretension, proteccion_cortocircuito, proteccion_sobrecarga, ripple_con_baterias, ripple_sin_baterias, regulador_diodos_carga, rango_salida_nicd, rango_salida_pbca
- alimentacion: tipo, **tension**, rango_tension, rango_frecuencia, conexion_neutro, corriente_cortocircuito, tipo_interruptor_acometida, potencia_transformador, corriente_conexion_transformador
- salida: tipo_interruptor_consumo, tipo_interruptor_baterias, sistema_rectificacion
- gabinete: material, acceso, grado_proteccion, tipo_pintura, color
- aparatos_medida: protocolo_comunicacion, puerto_comunicacion
- accesorios: panel_control, potencia_resistencias, tension_aislacion_cables, material_cables
- garantia: condiciones

**NUMERIC FIELDS that should keep = operator:**
- productos: tension_nominal, corriente_nominal
- especificaciones: temperatura_maxima, temperatura_minima, altura_snm, humedad_relativa_max, rendimiento_minimo, tension_flote_min, tension_flote_max, tension_fondo_min, tension_fondo_max, espesor_chapa, espesor_pintura
- alimentacion: **frecuencia** (this is INTEGER, not text)
- salida: **tension_nominal**, **corriente_nominal**, maxima_corriente_consumos (these are INTEGER, not text)
- gabinete: ancho, alto, profundidad, peso, espesor_chapa, espesor_pintura
- accesorios: tension_resistencias
- garantia: meses

**BOOLEAN FIELDS that should keep = operator:**
- All boolean fields (apto_pb_ac, apto_ni_cd, modo_manual_automatico, etc.)

**Examples:**
- Text field with ILIKE: `a.tipo ILIKE '%trifásica%'` → `similarity(a.tipo, 'trifásica') > 0.5`
- Text field with =: `p.marca = 'SERVELEC'` → `similarity(p.marca, 'SERVELEC') > 0.5`
- **Text field tension**: `a.tension = '380'` → `similarity(a.tension, '380') > 0.4` **SIMILARITY SCORE MUST BE 0.4** (tension in alimentacion is VARCHAR)
- **Numeric field tension_nominal**: `s.tension_nominal = 110` → **KEEP AS IS** (tension_nominal in salida is INTEGER)
- Numeric field: `s.corriente_nominal = 50` → **KEEP AS IS** (INTEGER)
- Numeric field: `a.frecuencia = 50` → **KEEP AS IS** (INTEGER)
- Boolean field: `e.apto_pb_ac = true` → **KEEP AS IS**

**Critical distinction:**
- `alimentacion.tension` = TEXT (use similarity)
- `salida.tension_nominal` = INTEGER (keep =)
- `productos.tension_nominal` = INTEGER (keep =)

**Important**: 
- Remove any wildcards (%) from values when using similarity function
- Only apply similarity to text/VARCHAR fields from the list above
- Keep = operator for all numeric and boolean fields
- Pay special attention: "tension" (text) vs "tension_nominal" (numeric)

### 3. English to Spanish Translation
Translate any English text values in the WHERE clause to Spanish:

- **Common translations**:
  - "Indoor" / "interior" → "Interior"
  - "Outdoor" / "exterior" → "Exterior"
  - "Three-phase" / "threephase" → "Trifásica"
  - "Single-phase" / "singlephase" / "monophase" → "Monofásica"
  - "Continuous" → "Continuo"
  - "Natural" → "Natural"
  - "Forced" → "Forzada"
  - "Manual" → "Manual"
  - "Automatic" → "Automático"
  - "Argentina" → "Argentina"
  - "Self-regulated" → "Autorregulado"

- Apply translation to any text value in ILIKE or = comparisons:
  - Original: `a.tipo ILIKE '%three-phase%'`
  - Optimized: `a.tipo ILIKE '%trifásica%'`
  
  - Original: `e.tipo_instalacion ILIKE '%indoor%'`
  - Optimized: `e.tipo_instalacion ILIKE '%Interior%'`

- **Important**: Always preserve the SQL syntax (ILIKE, wildcards %, quotes, etc.) - only translate the actual text content

### 4. Add origen_consulta Column
If the query does NOT already have `origen_consulta` in the SELECT clause, add it:

- **For single queries**: Add `1 AS origen_consulta` after `p.*`
  - Original: `SELECT DISTINCT p.*`
  - Optimized: `SELECT DISTINCT p.*, 1 AS origen_consulta`

- **For UNION ALL queries**: Add sequential numbering to each SELECT statement
  - Original:
    ```sql
    SELECT DISTINCT p.*
    FROM productos p
    WHERE ...
    UNION ALL
    SELECT DISTINCT p.*
    FROM productos p
    WHERE ...
    ```
  - Optimized:
    ```sql
    SELECT DISTINCT p.*, 1 AS origen_consulta
    FROM productos p
    WHERE ...
    UNION ALL
    SELECT DISTINCT p.*, 2 AS origen_consulta
    FROM productos p
    WHERE ...
    ```

- **If origen_consulta already exists**: Leave it as is, do not modify the numbering

### 5. Remove NULL Conditions
Remove any WHERE conditions that check for NULL values:

- **IS NULL comparisons**: Remove completely from WHERE clause
  - Original: `WHERE p.marca ILIKE '%SERVELEC%' AND s.corriente_nominal IS NULL`
  - Optimized: `WHERE p.marca ILIKE '%SERVELEC%'`

- **IS NOT NULL comparisons**: Remove completely from WHERE clause
  - Original: `WHERE p.codigo ILIKE '%RECT%' AND e.temperatura_maxima IS NOT NULL`
  - Optimized: `WHERE p.codigo ILIKE '%RECT%'`

- **= NULL comparisons**: Remove completely (these are incorrect SQL anyway)
  - Original: `WHERE p.tipo ILIKE '%Autorregulado%' AND s.tension_nominal = NULL`
  - Optimized: `WHERE p.tipo ILIKE '%Autorregulado%'`

- **Handle AND/OR logic**: When removing NULL conditions, clean up any resulting orphaned AND/OR operators
  - Original: `WHERE AND s.corriente_nominal IS NULL`
  - Optimized: Remove the entire WHERE clause if no other conditions remain
  
  - Original: `WHERE p.marca = 'SERVELEC' AND s.corriente_nominal IS NULL AND e.temperatura_maxima <= 45`
  - Optimized: `WHERE p.marca = 'SERVELEC' AND e.temperatura_maxima <= 45`

## Important Notes

1. **Replace ALL text comparisons with similarity** - Every text field comparison (ILIKE or =) must be converted to similarity function. Only numeric and boolean comparisons should remain with their operators.
2. **Only modify specific numeric comparison fields** - temperatura_maxima, temperatura_minima, humedad_relativa_max use range operators (<=, >=)
3. **Preserve query structure** - Keep all JOINs, SELECT clauses, and other conditions exactly as they are
4. **Maintain formatting** - Keep the same indentation and line breaks as the original query
5. **Handle multiple occurrences** - If any of these fields appear multiple times, apply the rule to ALL occurrences
6. **Case sensitivity in translations** - Use proper Spanish capitalization (e.g., "Trifásica" not "TRIFÁSICA")
7. **Always check for origen_consulta** - Add it if missing, preserve it if already present
8. **Remove all NULL checks** - Any condition checking for NULL values (IS NULL, IS NOT NULL, = NULL) must be completely removed from the WHERE clause
9. **Strip wildcards from similarity** - Remove % symbols from values when converting to similarity function
10. **Identify text vs numeric fields** - Text fields (VARCHAR, TEXT) use similarity; numeric fields (INTEGER, DECIMAL) keep = operator

## Response Format

Respond ONLY with the optimized SQL query, without explanations, without markdown code blocks, without text before or after. Just pure SQL.

## Examples

**Example 1 - Single Query:**

**Input Query:**
```sql
SELECT DISTINCT p.*
FROM productos p
LEFT JOIN alimentacion a ON p.id = a.producto_id
LEFT JOIN especificaciones e ON p.id = e.producto_id
LEFT JOIN salida s ON p.id = s.producto_id
WHERE a.tipo ILIKE '%three-phase%'
  AND a.tension = '380'
  AND e.temperatura_maxima = 45
  AND e.temperatura_minima = -10
  AND e.humedad_relativa_max = 85
  AND e.tipo_instalacion ILIKE '%indoor%'
  AND a.frecuencia = 50
  AND s.corriente_nominal = 50
```

**Expected Output:**
```sql
SELECT DISTINCT p.*, 1 AS origen_consulta
FROM productos p
LEFT JOIN alimentacion a ON p.id = a.producto_id
LEFT JOIN especificaciones e ON p.id = e.producto_id
LEFT JOIN salida s ON p.id = s.producto_id
WHERE similarity(a.tipo, 'trifásica') > 0.5
  AND similarity(a.tension, '380') > 0.5
  AND e.temperatura_maxima <= 45
  AND e.temperatura_minima >= -10
  AND e.humedad_relativa_max <= 85
  AND similarity(e.tipo_instalacion, 'Interior') > 0.5
  AND a.frecuencia = 50
  AND s.corriente_nominal = 50
```

Note: `s.corriente_nominal` and `a.frecuencia` are numeric, so they keep the = operator.

**Example 2 - UNION ALL Query:**

**Input Query:**
```sql
SELECT DISTINCT p.*
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE s.corriente_nominal = 50
  AND e.temperatura_maxima = 45
UNION ALL
SELECT DISTINCT p.*
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE s.corriente_nominal = 40
  AND e.temperatura_minima = -10
```

**Expected Output:**
```sql
SELECT DISTINCT p.*, 1 AS origen_consulta
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE s.corriente_nominal = 50
  AND e.temperatura_maxima <= 45
UNION ALL
SELECT DISTINCT p.*, 2 AS origen_consulta
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE s.corriente_nominal = 40
  AND e.temperatura_minima >= -10
```

**Example 3 - Query with NULL conditions:**

**Input Query:**
```sql
SELECT DISTINCT p.*
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE p.marca ILIKE '%SERVELEC%'
  AND s.corriente_nominal IS NULL
  AND e.temperatura_maxima = 45
  AND s.tension_nominal = NULL
```

**Expected Output:**
```sql
SELECT DISTINCT p.*, 1 AS origen_consulta
FROM productos p
LEFT JOIN salida s ON p.id = s.producto_id
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE similarity(p.marca, 'SERVELEC') > 0.5
  AND e.temperatura_maxima <= 45
```

Note: Both `s.corriente_nominal IS NULL` and `s.tension_nominal = NULL` were removed, and `p.marca ILIKE '%SERVELEC%'` was converted to similarity function.

**Example 4 - Query with origen_consulta already present:**

**Input Query:**
```sql
SELECT DISTINCT p.*, 1 AS origen_consulta
FROM productos p
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE e.temperatura_maxima = 45
```

**Expected Output:**
```sql
SELECT DISTINCT p.*, 1 AS origen_consulta
FROM productos p
LEFT JOIN especificaciones e ON p.id = e.producto_id
WHERE e.temperatura_maxima <= 45
```

Note: origen_consulta was already present, so only the temperatura_maxima operator was changed.

Now optimize the following SQL query:
{query}""")

query_scoring_prompt = ChatPromptTemplate.from_template("""

# System Prompt for SQL Query Scoring

You are an expert at adding match scoring logic to PostgreSQL queries. Your task is to wrap an existing SQL query with scoring calculations that evaluate how well each product matches the search criteria.

## Scoring Rules

### 1. The following SQL query has already been normalized.
All WHERE conditions must be transformed into score columns exactly as written.
Do NOT infer or add new conditions.

For each condition in the WHERE clause, create a corresponding score column:

**For similarity() comparisons:**
- Original: `similarity(a.tipo, 'trifásica') > 0.5`
- Score column: `CASE WHEN similarity(a.tipo, 'trifásica') > 0.5 THEN similarity(a.tipo, 'trifásica') ELSE 0 END AS score_tipo_alimentacion`

**For range comparisons (<=, >=):**
- Original: `e.temperatura_maxima <= 45`
- Score column: `CASE WHEN e.temperatura_maxima <= 45 THEN 1.0 ELSE 0.0 END AS score_temperatura_maxima`

**For exact numeric comparisons (=):**
- Original: `s.corriente_nominal = 50`
- Score column: `CASE WHEN s.corriente_nominal = 50 THEN 1.0 ELSE 0.0 END AS score_corriente_nominal`

**For boolean comparisons:**
- Original: `e.apto_pb_ac = true`
- Score column: `CASE WHEN e.apto_pb_ac = true THEN 1.0 ELSE 0.0 END AS score_apto_pb_ac`

### 2. Field Weights

Apply these weights when calculating the total match score:

**Critical fields (weight 0.20):**
- s.corriente_nominal
- s.tension_nominal
- a.tipo (tipo de alimentación)
- a.tension

**Important fields (weight 0.15):**
- p.tipo
- e.tipo_instalacion

**Desirable fields (weight 0.10):**
- a.frecuencia
- e.temperatura_maxima
- e.temperatura_minima
- e.humedad_relativa_max

**Other fields (weight 0.05):**
- Any other condition not listed above

### 3. Output Structure

CRITICAL: You MUST wrap the scored query in a subquery to filter by match_score_total:

```sql
SELECT *
FROM (
    SELECT DISTINCT p.*, 
        [origen_consulta value] AS origen_consulta,
        [score columns here],
        (
            [weighted score calculation here]
        ) AS match_score_total
    FROM productos p
    [LEFT JOINs from original query]
    [Remove the original WHERE clause completely]
) q
WHERE q.match_score_total > 0.5
ORDER BY q.match_score_total DESC, q.origen_consulta
LIMIT 50;
```

### 4. Handle UNION ALL queries

**CRITICAL**: When the original query contains UNION ALL, ALL queries must have the EXACT SAME columns.

Rules:
1. Identify ALL unique conditions across ALL UNION queries
2. Create score columns for ALL conditions in EVERY query
3. If a condition doesn't exist in a specific query, set its score to 0

Example of WRONG approach (different columns):
```sql
-- Query 1 has score_corriente_nominal
SELECT *, score_corriente_nominal, match_score_total FROM (...) q1
UNION ALL
-- Query 2 has score_tension - WRONG! Different columns
SELECT *, score_tension, match_score_total FROM (...) q2
```

Example of CORRECT approach (same columns):
```sql
-- Query 1: has corriente_nominal condition, no tension condition
SELECT * FROM (
    SELECT DISTINCT p.*, 
        1 AS origen_consulta,
        CASE WHEN s.corriente_nominal = 50 THEN 1.0 ELSE 0.0 END AS score_corriente_nominal,
        0 AS score_tension,  -- Not in this query's conditions, but must exist
        (
            0.20 * CASE WHEN s.corriente_nominal = 50 THEN 1.0 ELSE 0.0 END +
            0 * 0  -- tension weight but score is 0
        ) AS match_score_total
    FROM productos p
    LEFT JOIN salida s ON p.id = s.producto_id
) q1
WHERE q1.match_score_total > 0.5
UNION ALL
-- Query 2: has tension condition, no corriente_nominal condition
SELECT * FROM (
    SELECT DISTINCT p.*, 
        2 AS origen_consulta,
        0 AS score_corriente_nominal,  -- Not in this query's conditions, but must exist
        CASE WHEN a.tension = '380' THEN 1.0 ELSE 0.0 END AS score_tension,
        (
            0 * 0 +  -- corriente_nominal weight but score is 0
            0.20 * CASE WHEN a.tension = '380' THEN 1.0 ELSE 0.0 END
        ) AS match_score_total
    FROM productos p
    LEFT JOIN alimentacion a ON p.id = a.producto_id
) q2
WHERE q2.match_score_total > 0.5
ORDER BY match_score_total DESC, origen_consulta
LIMIT 50;
```

**Step-by-step process for UNION ALL:**
1. Scan ALL queries in the UNION ALL
2. Extract ALL unique conditions across all queries
3. Create a complete list of score column names
4. For each query:
   - Add score columns for its actual conditions
   - Add `0 AS score_[field]` for conditions that don't exist in this query
   - Include all condition scores in match_score_total calculation (even if 0)
5. Ensure EVERY query has the EXACT same column list in the EXACT same order

**Column order must be:**
1. All columns from `p.*`
2. `origen_consulta`
3. `match_score_total`

### 5. Important Notes

1. **Remove the original WHERE clause** - All conditions become score calculations
2. **Use subquery alias** - The subquery must have an alias (e.g., `q`, `q1`, `q2`, etc.)
3. **Filter in outer query** - Apply `WHERE match_score_total > 0.5` in the outer SELECT
4. **Preserve all columns** - Keep all original columns from `p.*` and `origen_consulta`
5. **Score naming** - Name score columns as `score_[field_name]` (e.g., `score_corriente_nominal`)
6. **Calculate total weight** - Sum of all weights should equal 1.0 (adjust proportionally if needed)

## Response Format

Respond ONLY with the complete SQL query, without explanations, without markdown code blocks, without text before or after. Just pure SQL.


```



Now add scoring logic to the following SQL query:
{query}""")

# Configurar chains
classification_chain = classification_prompt | llm | StrOutputParser()
extraction_chain = extraction_prompt | llm.with_structured_output(SistemaCargadorRectificador)
multi_item_extraction_chain = multi_item_extraction_prompt | llm.with_structured_output(LicitacionCompleta)
query_generator_chain = (
    query_generator_prompt 
    | llm 
    | StrOutputParser()
    | RunnableLambda(lambda x: {"query": x})
    | query_sanitizer_prompt
    | fast_llm 
    | StrOutputParser()
    |RunnableLambda(lambda x: {"query": x})
    | query_scoring_prompt
    | llm 
    | StrOutputParser()
)

In [ ]:
# ============================================================
# FUNCIONES PARA GENERAR QUERY
# ============================================================


def generate_query(json_response: dict) -> str:
    result = query_generator_chain.invoke({"json_input": json_response})
    return result

In [ ]:
# ============================================================
# EJECUCIÓN - Lectura y chunking del PDF
# ============================================================

pages_data = read_pdf_from_file(file)
chunks = chunk_text(None, pages_data)

print(f"Total chunks generados: {len(chunks)}")

In [ ]:
# ============================================================
# EJECUCIÓN - Clasificación de chunks
# ============================================================

for chunk in chunks:
    classification = classification_chain.invoke({"text_chunk": chunk['page_content']})
    chunk['classification'] = classification

technical_count = sum(1 for c in chunks if c['classification'] == 'TECHNICAL')
print(f"Chunks técnicos: {technical_count}/{len(chunks)}")

In [ ]:
# ============================================================
# EJECUCIÓN - Extracción estructurada de datos técnicos
# ============================================================

technical_chunks = [chunk for chunk in chunks if chunk['classification'] == 'TECHNICAL']

for technical_chunk in technical_chunks:
    try:
        structured_data = extraction_chain.invoke({"text_chunk": technical_chunk['page_content']})
        technical_chunk['structured_data'] = structured_data
    except Exception as e:
        technical_chunk['structured_data'] = None
        logging.error(f"Error processing chunk {technical_chunk['metadata']}: {e}")

print(f"Procesados {len(technical_chunks)} chunks técnicos")

In [ ]:
# ============================================================
# EJECUCIÓN - Generación del reporte de resultados
# ============================================================

answer = ""
for chunk in technical_chunks:
    answer += f"Document page {chunk['metadata']['page_number']} chunk {chunk['metadata']['chunk_index']}\n"
    answer += f"Document content: {chunk['page_content']}\n"
    answer += f"Extracted data: {chunk['structured_data']}\n"
    answer += "--------------------------------------------------\n\n"
file_path = os.path.join(os.path.dirname(os.getcwd()), 'servelec-final-project', 'results', f'report_{DOCUMENT_ID}.json')
os.makedirs(os.path.dirname(file_path), exist_ok=True)
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(answer, f, ensure_ascii=False, indent=2, default=str)
print(f"Reporte generado: {len(answer)} caracteres")

In [ ]:
# ============================================================
# EJECUCIÓN - Guardado de resultados (método Langchain)
# ============================================================

file_path = os.path.join(os.path.dirname(os.getcwd()), 'servelec-final-project', 'results', f'{DOCUMENT_ID}.json')

results = {
    "document_id": DOCUMENT_ID,
    "method": "langchain_chunking_classification",
    "total_chunks": len(chunks),
    "technical_chunks": len(technical_chunks),
    "extraction_results": []
}

for chunk in technical_chunks:
    chunk_result = {
        "page_number": chunk['metadata']['page_number'],
        "chunk_index": chunk['metadata']['chunk_index'],
        "content": chunk['page_content'],
        "structured_data": chunk['structured_data'].model_dump() if chunk['structured_data'] else None
    }
    results["extraction_results"].append(chunk_result)

os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2, default=str)

print(f"Guardado en: {file_path}")

## Llamada a Gemini File Search

In [ ]:
# ============================================================
# SETUP GEMINI FILE SEARCH
# ============================================================

gemini_model = genai.GenerativeModel("gemini-2.5-flash")

gemini_extraction_prompt = """
You are an expert electrical and electronics tender document analyzer.
Your task is to extract ALL electrical products and equipment mentioned
in this document with their complete specifications.

**Instructions:**
1. Read the entire document carefully
2. Identify every electrical product, equipment, or system mentioned
3. For each item, extract ALL available specifications and technical details
4. Include product names, models, brands, technical parameters, quantities, and any other relevant information
5. Be thorough and comprehensive - do not miss any items
6. Be flexible in recognizing different terminologies for similar products
7. In the texts provided, some items may be described in detail while others may be briefly mentioned - extract whatever information is available and add that flexibility to your extraction

**Categories to look for:**
- Electrical equipment (transformers, motors, generators, etc.)
- Power systems (UPS, batteries, rectifiers, chargers, etc.)
- Protection devices (circuit breakers, fuses, relays, etc.)
- Control systems and panels
- Cables and wiring systems
- Measurement and monitoring equipment
- Installation materials and accessories
- Any other electrical/electronic components

**For each item extract:**
- Product name and type
- Brand and model (if available)
- Technical specifications (voltage, current, power, frequency, capacity, etc.)
- Physical specifications (dimensions, weight, materials, etc.)
- Environmental conditions (temperature, humidity, altitude, etc.)
- Standards and certifications
- Quantity required
- Any special requirements or conditions

**Output format:**
Provide a comprehensive, structured list of all products with their complete
specifications. Be thorough and detailed. Include everything mentioned in the
document.

Analyze the document now and extract all electrical products and equipment
with their specifications.
"""

In [ ]:
# ============================================================
# EJECUCIÓN - Extracción con Gemini
# ============================================================

print("Subiendo PDF a Gemini...")
uploaded_file = genai.upload_file(path=file, display_name=f"{DOCUMENT_ID}.pdf")

while uploaded_file.state.name == "PROCESSING":
    time.sleep(2)
    uploaded_file = genai.get_file(uploaded_file.name)

if uploaded_file.state.name == "FAILED":
    raise ValueError(f"Error al procesar el archivo: {uploaded_file.state}")

print("Extrayendo productos...")
gemini_response = gemini_model.generate_content([gemini_extraction_prompt, uploaded_file])
gemini_extracted_text = gemini_response.text

print(f"✓ Extraídos {len(gemini_extracted_text)} caracteres")

In [ ]:
# ============================================================
# EJECUCIÓN - Normalización de respuesta Gemini (Multi-Item)
# ============================================================

print("Normalizando respuesta de Gemini...")

try:
    gemini_structured_data = multi_item_extraction_chain.invoke({"text_chunk": gemini_extracted_text})
    print(f"✓ Extraídos {len(gemini_structured_data.items)} items")
except Exception as e:
    gemini_structured_data = None
    logging.error(f"Error al normalizar respuesta de Gemini: {e}")
    print(f"Error: {e}")

In [ ]:
# ============================================================
# EJECUCIÓN - Guardado resultados Gemini (Multi-Item)
# ============================================================

gemini_file_path = os.path.join(os.path.dirname(os.getcwd()), 'servelec-final-project', 'results', 'gemini', f'{DOCUMENT_ID}_multi_item.json')
gemini_results = gemini_structured_data.model_dump() if gemini_structured_data else None

os.makedirs(os.path.dirname(gemini_file_path), exist_ok=True)

with open(gemini_file_path, "w", encoding="utf-8") as f:
    json.dump(gemini_results, f, ensure_ascii=False, indent=2, default=str)

print(f"Guardado en: {gemini_file_path}")

In [ ]:
# ============================================================
# COMPARACIÓN DE RESULTADOS
# ============================================================

multi_item_path = os.path.join(os.path.dirname(os.getcwd()), 'servelec-final-project', 'results', 'gemini', f'{DOCUMENT_ID}_multi_item.json')

if os.path.exists(multi_item_path):
    with open(multi_item_path, encoding='utf-8') as f:
        multi_item_data = json.load(f)

    print(f"\nItems extraídos: {len(multi_item_data['items'])}")
    for item in multi_item_data['items']:
        desc = item['descripcion'][:50] + "..." if len(item['descripcion']) > 50 else item['descripcion']
        print(f"  {item['numero_item']}. {desc} (x{item['cantidad']})")
else:
    print("Ejecutar celdas de Gemini primero")

In [ ]:
# ============================================================
# Generacion de querys
# ============================================================
print(f'Procesando archivo JSON: {json_file}')
with open(json_file, 'r', encoding='utf-8') as f:
    json_content = f.read()
generate_query_result = generate_query(json_content)
print(generate_query_result)